### Collect codes of public elementary schools in municipality of Madrid.

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pickle
import dryscrape
from time import sleep
from random import randint

In [ ]:
# Url to perform advanced searches.
url_advsearch = "http://www.madrid.org/wpad_pub/run/j/BusquedaAvanzada.icm"

In [ ]:
# Parameter for searching public primary schools in municipality of Madrid
params = {"titularidadPublica": "S", "cdMuni": "079", "cdNivelEdu": "6545"}

In [ ]:
# Request and parse list of schools.
schools = BeautifulSoup(requests.post(url_advsearch, data = params).content, "lxml")

In [ ]:
# Extract list of school codes.
school_codes = schools.findAll(attrs = {"name": "codCentrosExp", "value": re.compile("^.+$")})[0]["value"]

In [ ]:
# Convert from string to list.
school_codes = school_codes.split(";")

# Check codes.
# 247 it's ok.
print len(school_codes)

In [ ]:
# Save list of school codes
with open("school_codes.txt", "wb") as f:
    pickle.dump(school_codes, f)

In [ ]:
# Load list of school codes
#with open("school_codes.txt", "rb") as f:
#    school_codes = pickle.load(f)

### Extract tables from school cards.

In [ ]:
# School card url.
url_schoolcard = "http://www.madrid.org/wpad_pub/run/j/MostrarFichaCentro.icm"

# School code parameter.
school_code_par = "cdCentro="

# List of schools urls
schools_urls = [url_schoolcard + "?" + school_code_par + code for code in school_codes]

In [ ]:
# Collect data.

school_tables_collection = {}
sts = 0
nsts = 0

for z, school in enumerate(schools_urls):
    try:
        render = dryscrape.Session()
        render.visit(school)
        render.exec_script('document.getElementById("nivEd12.grafica3").click();')
        source = render.body()
        school_card = BeautifulSoup(source, "lxml")
        school_tables = school_card.findAll('table', class_="tablaGraficaDatos")
        school_name = school_card.find(style="text-transform:uppercase").next.next
        for i, table in list(enumerate(school_tables)):
            if i <= 1:
                school_tables_collection[school_name + "_" + str(i)] = pd.read_html(table.prettify())
        sts += 1
        !echo "Tables of school {schools_urls[z]} extracted" >> scrape_log.txt
        print '{0}\r'.format("Schools with statistics: %i, Schools without statistics: %i, Total: %i" % (sts, nsts, sts + nsts)),
        render.reset()
        sleep(randint(1, 4))
    except:
        nsts += 1
        !echo "School {schools_urls[z]} doesn't have statistics" >> scrape_log.txt
        print '{0}\r'.format("Schools with statistics: %i, Schools without statistics: %i, Total: %i" % (sts, nsts, sts + nsts)),
        sleep(randint(1, 4))

In [ ]:
# Save dictionary with tables
with open("school_tables_collection.txt", "wb") as f:
    pickle.dump(school_tables_collection, f, protocol=pickle.HIGHEST_PROTOCOL)